#  High‑Level Flow

- Prepare data (CSV; label in first column) and upload to S3.

- Create an Estimator with the XGBoost container + hyperparams.

- Train (fit) on train/validation data.

- Deploy a real‑time HTTPS endpoint (optional if you only need batch predictions).

- Infer from any notebook or client that has AWS credentials.

- Monitor & Optimize costs; Clean up (delete the endpoint) when done.


## Minimal Architecture

- Notebook / Studio orchestrates everything via SageMaker SDK.

- S3 stores training data & model artifacts.

- SageMaker Training spins a training job using XGBoost container.

- SageMaker Hosting runs a managed endpoint for real‑time inference.


## Prerequisites & Setup

- AWS account with SageMaker access in your region (e.g., ap-southeast-1).

- Execution role with permissions: SageMaker, S3 read/write (see IAM sample below).

- Python deps: `sagemaker`, `boto3`, `pandas`, `scikit-learn`.

In [1]:
import boto3, sagemaker, time, os
from sagemaker import Session
from sagemaker import image_uris
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer
from sagemaker import get_execution_role

session = Session()
region = boto3.Session().region_name or "ap-southeast-1"   # SG default if none
try:
    role = get_execution_role()  # works inside SageMaker
except Exception:
    role = "arn:aws:iam::<YOUR-AWS-ACCOUNT-ID>:role/<YOUR-SAGEMAKER-EXECUTION-ROLE>"  # if running outside

# Pick or create a bucket
bucket = session.default_bucket()  # or set your own existing bucket name
prefix = "iris-xgb"


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
role

'arn:aws:iam::215470142970:role/service-role/AmazonSageMaker-ExecutionRole-20250816T210398'

## Load DATSET, split, and upload to S3

In [4]:
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Series(iris.target, name="label")

df = pd.concat([y, X], axis=1)  # label first

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df["label"])
val_df, test_df  = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df["label"])

os.makedirs("data", exist_ok=True)

train_path, val_path, test_path = "data/train.csv", "data/validation.csv", "data/test.csv"
train_df.to_csv(train_path, header=False, index=False)
val_df.to_csv(val_path, header=False, index=False)
test_df.to_csv(test_path, header=False, index=False)

s3_train = session.upload_data(train_path, bucket=bucket, key_prefix=f"{prefix}/data")
s3_val   = session.upload_data(val_path,   bucket=bucket, key_prefix=f"{prefix}/data")
s3_test  = session.upload_data(test_path,  bucket=bucket, key_prefix=f"{prefix}/data")
s3_output = f"s3://{bucket}/{prefix}/output"
print(s3_train, s3_val, s3_test)

s3://sagemaker-ap-southeast-1-215470142970/iris-xgb/data/train.csv s3://sagemaker-ap-southeast-1-215470142970/iris-xgb/data/validation.csv s3://sagemaker-ap-southeast-1-215470142970/iris-xgb/data/test.csv


## Create the XGBoost Estimator

### Estimator Config
- image_uri = “Which code container runs?”

- role = “What AWS permissions does it have?”

- instance_type/count = “On what hardware does it run?”

- output_path = “Where do results go?”

- set_hyperparameters = “What arguments are passed to the containerized algorithm?”

In [5]:
from sagemaker.estimator import Estimator

xgb_image = image_uris.retrieve(framework="xgboost", region=region, version="1.7-1")  # stable version

estimator = Estimator(
    image_uri=xgb_image,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=s3_output,
    sagemaker_session=session
)

# Hyperparameters for multiclass classification
estimator.set_hyperparameters(
    objective="multi:softprob",  # returns class probabilities
    num_class=3,
    eval_metric="mlogloss",
    num_round=200,
    max_depth=4,
    eta=0.2,
    subsample=0.8,
    colsample_bytree=0.8
)


## Train

In [6]:
from sagemaker.inputs import TrainingInput

train_input = TrainingInput(s3_data=s3_train, content_type="text/csv")
val_input   = TrainingInput(s3_data=s3_val,   content_type="text/csv")

estimator.fit({"train": train_input, "validation": val_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2025-09-27-11-59-41-157


2025-09-27 11:59:43 Starting - Starting the training job...
2025-09-27 11:59:57 Starting - Preparing the instances for training...
2025-09-27 12:00:18 Downloading - Downloading input data...
2025-09-27 12:01:08 Downloading - Downloading the training image......
2025-09-27 12:02:14 Training - Training image download completed. Training in progress.../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-09-27 12:02:19.044 ip-10-0-83-228.ap-southeast-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-09-27 12:02:19.126 ip-10-0-83-228.ap-southeast-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-09-27:12:02:19:INFO] Imported framew

## Deploy HTTP Endpoints

In [9]:
predictor = estimator.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large"
)

# Set serializer/deserializer for CSV in, JSON out
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-09-27-12-06-26-368
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2025-09-27-12-06-26-368
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2025-09-27-12-06-26-368


------!

In [8]:
predictor.delete_endpoint()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 predictor.delete_endpoint()                                                                  │
│   2                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predictor' is not defined

## Run inference (Run this in a separate notebook)

In [ ]:
import boto3
import sagemaker
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

session = sagemaker.Session()

predictor = sagemaker.Predictor(
    endpoint_name="sagemaker-xgboost-2025-08-18-07-13-45-645", 
    sagemaker_session=session
)

# Set serializers again (these don’t persist across notebooks)
predictor.serializer = CSVSerializer()
predictor.deserializer = JSONDeserializer()


In [ ]:
# Example: single row (sepal length, sepal width, petal length, petal width)
row = [[5.1, 3.5, 1.4, 0.2]]

payload = ",".join(map(str, row[0]))
result = predictor.predict(payload)

print(result)
# → {'predictions': [{'score': [0.98, 0.01, 0.01]}]}


 ## Hosting the 

In [12]:
s3_output

's3://sagemaker-ap-southeast-1-215470142970/iris-xgb/output'

In [14]:
from sagemaker.model import Model
from sagemaker import image_uris
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer

# fill these from the console Model details
model_data = "s3://sagemaker-ap-southeast-1-215470142970/iris-xgb/output/sagemaker-xgboost-2025-09-27-11-59-41-157/output/model.tar.gz"
xgb_image  = image_uris.retrieve(framework="xgboost", region=region, version="1.7-1")

model = Model(
    image_uri=xgb_image,
    model_data=model_data,
    role=role,
    sagemaker_session=session,
    name="iris-xgb-model"
)

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.large",
    endpoint_name="iris-xgb-rt"
)
predictor.serializer   = CSVSerializer()
predictor.deserializer = CSVDeserializer()


INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating model with name: iris-xgb-model
INFO:sagemaker:Creating endpoint-config with name iris-xgb-rt
INFO:sagemaker:Creating endpoint with name iris-xgb-rt


------!

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   instance_type="ml.m5.large",                                                            │
│   21 │   endpoint_name="iris-xgb-rt"                                                             │
│   22 )                                                                                           │
│ ❱ 23 predictor.serializer   = CSVSerializer()                                                    │
│   24 predictor.deserializer = CSVDeserializer()                                                  │
│   25                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'serializer'